## Raster input for itzi model

1. DEM, http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_DEM/
2. Manning's friction map, 
3. water mask data, https://www.researchgate.net/publication/365035833_Global_Land_Water_Mask_Map_with_03_arc_sec_10_m_Resolution
4. extent shape in vector

In [3]:
%load_ext dotenv
%dotenv
import os
workdata_path=os.environ['data_work']
workdata_path

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


'/home/bulbul/Documents/work_data/'

In [5]:
import geopandas as gp
from shapely.geometry import box
import pandas as pd


db=gp.read_file(f'{workdata_path}/ea_shapefiles/ea_ghcf_icpac.shp')
xmin,ymin,xmax,ymax =  db.total_bounds

geom =box(*db.total_bounds)

#db=pd.DataFrame()
#db['geometry']=[geom]

#db1=gp.GeoDataFrame(db)

#db1.to_file('f'{workdata_path}/ea_shapefiles/ea_extent_boundary.shp')
#db1.plot()

In [10]:
import rioxarray

xds = rioxarray.open_rasterio(f'{workdata_path}flood_itzi/mannings/floodplains_12D.nc',masked=True)

#xds1= xds.rio.clip(geom)
xds1=xds.rio.write_crs(4326, inplace=True)
xds2= xds1.rio.clip([geom])

In [15]:
#xds2['floodplains_roughness'].plot()

xds2.floodplains_roughness.rio.to_raster(f'{workdata_path}flood_itzi/mannings/manning_ea.tiff')

In [18]:
import rioxarray
from rioxarray import merge
import glob

occ_tiffs=glob.glob(f'{workdata_path}flood_itzi/water_mask/occurrence_*.tif')

# elements = []

# for val in occ_tiffs:
#     elements.append(rioxarray.open_rasterio(val))
    
# merged = merge.merge_arrays(elements, nodata=0.0)

xds=rioxarray.open_rasterio(val)
xds

<xarray.DataArray (band: 1, y: 40000, x: 40000)>
[1600000000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 20.0 20.0 20.0 20.0 20.0 ... 30.0 30.0 30.0 30.0
  * y            (y) float64 10.0 10.0 9.999 ... 0.000625 0.000375 0.000125
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     value

## subsetting water mask data

In [27]:
#import fiona
import rasterio
from rasterio.mask import mask
import geopandas as gp

nReserve = gpd.read_file('/home/ea_extent_boundary.shp')

#nReserve_proj = nReserve.to_crs({'init': 'epsg:32633'})


#with fiona.open('/home/ea_extent_boundary.shp', "r") as shapefile:
#    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open("/home/landWater2020.tif") as src:
    out_image, transformed = mask(src, nReserve.geometry, crop=True)
    out_profile = src.profile.copy()

out_profile.update({'width': out_image.shape[2],'height': out_image.shape[1], 'transform': transformed})
with rasterio.open("/home/ea_landwater2020.tif", 'w', **out_profile) as dst:
    dst.write(out_image)

MemoryError: Unable to allocate 138. GiB for an array with shape (418691, 354922) and data type uint8

In [24]:
import rasterio as rio
import geopandas as gpd
from rasterio.mask import mask
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling

nReserve = gpd.read_file('/home/ea_extent_boundary.shp')

#nReserve_proj = nReserve.to_crs({'init': 'epsg:4326'})




# with rasterio.open(filepath) as src:
#     rst = src.read(1, window=from_bounds(xmin,ymin,xmax,ymax, src.transform))


with rio.open("/home/landWater2020.tif") as src:
    out_image, out_transform = mask(src, nReserve.geometry,crop=True, filled=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rio.open("/home/ea_landwater2020.tif", "w", **out_meta) as dest:
    dest.write(out_image)
    
# image=out_image.astype(rasterio.uint8)
    
# with rasterio.open(
#         '/home/landWater2020.tif', 'w',
#         driver='GTiff', width=out_image.shape[2], height=out_image.shape[1], count=1,
#         dtype=image.dtype) as dst:
#     dst.write(image, window=from_bounds(xmin,ymin,xmax,ymax, src.transform), indexes=1)

MemoryError: Unable to allocate 138. GiB for an array with shape (418691, 354922) and data type uint8

In [13]:
out_meta

NameError: name 'out_meta' is not defined